In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

In [2]:
model = read_sbml_model('data/models/iMM904.xml')
model

Name,iMM904
Memory address,0x07fca38fee1d0
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [3]:
# Reaction 1 (HA-TAL):L-tyrosine <=> (E).4.coumarate +   NH4+
new_reaction1 = Reaction('HA-TAL') # 3-(o)xopropanoate:(N)ADP+ (o)xidoreductase
coumarate4 = Metabolite(id='coumarate4', compartment='c')
new_reaction1.add_metabolites({model.metabolites.tyr__L_c: -1, # Search L-tyrisine ID
                               coumarate4: 1, # 3-Oxopropanoate
                               model.metabolites.nh4_c: 1, # Search fpr NH4 ID
                              })
model.add_reactions([new_reaction1])

In [4]:
new_reaction2 = Reaction('AT-4CL1')
E4_coumaroyl_CoA = Metabolite(id='e4_coumaroyl_coa', compartment='c')
new_reaction2.add_metabolites({model.metabolites.coumarate4: -1,
                               model.metabolites.atp_c: -1,
                               model.metabolites.coa_c: -1,
                               E4_coumaroyl_CoA: 1,
                               model.metabolites.amp_c: 1,
                               model.metabolites.ppi_c: 1,
                              })
model.add_reactions([new_reaction2])

In [5]:
new_reaction3 = Reaction('VVVST1')
trans_resv = Metabolite(id='trans_resv', compartment='c')
new_reaction3.add_metabolites({model.metabolites.e4_coumaroyl_coa: -1,
                               model.metabolites.h_c: -3,
                               model.metabolites.malcoa_c: -3,
                               model.metabolites.co2_c: 4,
                               model.metabolites.coa_c: 4,
                               trans_resv: 1
                              })
model.add_reactions([new_reaction3])

In [7]:
model.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x07fca383786a0
Stoichiometry,e4_coumaroyl_coa + 3 h_c + 3 malcoa_c --> 4 co2_c + 4 coa_c + trans_resv + 3 H+ + 3 Malonyl CoA C24H33N7O19P3S --> 4 CO2 CO2 + 4 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [31]:
from cobra.io import write_sbml_model
write_sbml_model(model,'data/models/yeast_lyco.xml')

#### how much resveratrol is produced?

In [8]:
model.add_boundary(model.metabolites.trans_resv, type='demand')

Reaction identifier,DM_trans_resv
Name,demand
Memory address,0x07fca38378ef0
Stoichiometry,trans_resv --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [9]:
with model:
    model.objective=model.reactions.VVVST1
    lets_produce_resv= model.optimize().objective_value
lets_produce_resv

1.1785714285714293